In [8]:
using LinearAlgebra
using SwitchOnSafety
using ControlSystems
using StaticArrays
using HybridSystems
using JuMP
using MosekTools
using Random
using LightGraphs
using IncGammaBeta
using Arpack

In [9]:
CJSR = .5
under_JSR = 1.5

A1 = [-0.12264385232852724 -0.5132986766380205; -0.512239944583497 0.7657920763290312] * CJSR
A2 = [-1.0025775249390196 0.35564172966144936; -0.01094819510863365 0.5106100376225976] * under_JSR
Σ = [A1, A2]

@show A1
@show A2

G = LightAutomaton(2)
add_transition!(G, 1, 1, 1)
add_transition!(G, 1, 2, 2)
add_transition!(G, 2, 1, 1)
hs = discreteswitchedsystem(Σ, G)
nothing

A1 = [-0.06132192616426362 -0.25664933831901027; -0.2561199722917485 0.3828960381645156]
A2 = [-1.5038662874085293 0.533462594492174; -0.016422292662950474 0.7659150564338963]


In [ ]:
include("../../../src/WhiteBox.jl")
ub = white_box_CJSR_upper_bound(hs, 6)
@show ub
nothing

In [10]:
VERBOSE = false

include("../../../src/Trajectories.jl")
include("../../../src/Bounds.jl")

N = 10000
N_step = 100
N_begin = 100
N_range = N_begin:N_step:N

# Known information
V = nstates(hs.automaton)
m = size(Σ)[1]

# Simulations
u, v, x, y = generate_trajectories(Σ, G, N, 1)
y = reshape(y, size(x))

# Parameters
β = .99
β1 = (β + 1.) / 2.
β2 = β1

f = open("res_MQLF.txt", "w")
println("Simulations start")
for n = N_range
    time = @elapsed lower_bound, upper_bound = bounds_MQLF(x[:, 1:n], u[1:n], y[:, 1:n], v[1:n], V, β1, β2, m)
    println(f, "$lower_bound $upper_bound")
    if VERBOSE println("(n = $n) done (in $time s): [$lower_bound, $upper_bound]") end
end
close(f)


Simulations start


In [11]:
VERBOSE = false

include("../../../src/Bounds.jl")
l = 1

λ, _ = eigs(adjacency_matrix(hs.automaton.G), nev = 1, which = :LM)
quantity = V * λ[1]^l

u, v, x, y = generate_trajectories(Σ, G, N, l)
y = reshape(y, size(x))                         # to change if l ≂̸ 1

f = open("res_CQLF.txt", "w")
println("Simulations start")
for n = N_range
    time = @elapsed upper_bound = upper_bound_CQLF(x[:, 1:n], y[:, 1:n], β, l, quantity)
    println(f, "$upper_bound")
    if VERBOSE @show upper_bound end
end

close(f)

Simulations start


In [12]:
# N = 10000
# N_step = 100
# N_begin = 100
# N_range = N_begin:N_step:N
# CJSR = .5
# under_JSR = 1.5

using DelimitedFiles
data_CQLF = readdlm("res_CQLF.txt")
data_MQLF = readdlm("res_MQLF.txt")

keep = map(x -> x != -1, data_CQLF)
keep = reshape(keep, size(N_range))

using PyPlot
figure()
margins(x=0)
fill_between(N_range, ones(size(N_range)), color="grey", alpha = 0.3, label="Stability zone")
axhline(CJSR, linestyle="--", color="k", linewidth = 0.6, label="\$\\rho(G, \\Sigma)\$")
axhline(under_JSR, linestyle="-.", color="k", linewidth = 0.6, label="\$\\rho(\\Sigma)\$")
plot(N_range, data_MQLF[:, 1], "-", label="MQLF lower bound")
plot(N_range[keep], data_CQLF[keep], "-",  label="CQLF upper bound (β = 99%)")
plot(N_range, data_MQLF[:, 2], "-", label="MQLF upper bound (β = 99%)")
xlabel("Number of observations \$N\$")
xscale("log")
ylim((0, 3))
# yscale("log")
legend()
savefig("comparaison_third_case.pdf")